# Backpack for Learning Rust

There are many tips and tricks I've been collecting along the way
on my Rust journey. These are bits of knowledge that aren't 
written down in books or any single place. They're an accumulation
of things I've seen in forums, and StackOverflow posts, and
thing buried deep in the official docs.

I'm intending this page to be a "living" post. I'll keep adding
to it as I discover stuff.

## Easily add dependencies to `Cargo.toml` with cargo-add

Install it with `$ cargo install cargo-add`. This gives you a new cargo subcommand to call in any of your projects when you want to add a new dependency. It automatically adds that dependency to your `Cargo.toml` for you. The great thing is that you don't need to know the version string: it figures that out too.

Example:

```shell
$ cargo add fstrings
    Updating 'https://github.com/rust-lang/crates.io-index' index
      Adding fstrings v0.2.3 to dependencies
```

## Offline documentation

You can view offline documentation for both your dev project as well as all the Rust project documentation. Offline documentation can be very useful when commuting or travelling, or sitting on the beach.

As long as you have the `rustup` docs installed, you can immediately open a browser window to view the local documentation here:

```shell
$ rustup doc
```

As for documentation for your project, it can be generated like this, as long as all the dependency crates have already all be downloaded:

```shell
$ cargo doc --open
```
The `--open` parameter will open a browser window for you. Note that this documentation includes not only your own project, but also the docs of all your dependency crates.

## Simple error handling

While you're learning Rust, most of your programs are probably going to be command-line programs, i.e., applications. I strongly suggest you use a crate called `anyhow` to help you deal with error handling. Rust is quite pedantic about types and it can be a frustrating experience trying to figure out exactly how to deal with error types.

If you never want to deal with error types, you could just call `.unwrap()` on every `Result` type; however, this is a bad habit to get into, even when you're just learning. For one thing, it means that later you will have to go back and fix up all the `.unwrap()` calls if you get to the point where you want to clean up your program. But the more unfortunate thing is that you never get to learn how to deal with errors correctly.

This is the dilemma:
- You want to avoid calling `.unwrap()`
- You also want to avoid writing `match` statements every single time some function returns a `Result` type.

What to do? This scenario is described in The Book, where the `?` operator is introduced. Within a function that returns `Result`, if you call another function that returns `Result` you can use `?` to automatically either get the `Ok()` value, or propagate the `Err` up to the caller. That's wordy, but this is all I mean:

```rust
fn my_func() -> Result<(), ...> {
    let x = some_other_func()?;   
    Ok(())
}
```

In the example above, `some_other_func` returns a `Result`. If that `Result` is an `Ok` enum, the value will be assigned to `x`; but if the `Result` is an `Err` enum, that will be returned from `my_func` (and `x` will never have been assigned.

This `?` is great, but there's a big problem: the ellipses I used for the error type returned by `my_func` above is quite complicated to deal with if it doesn't match the `Err` type that `some_other_func()` can return. If the two error types are incompatible, the Rust compiler will complain. 

If you use `anyhow`, you won't have to worry about error types. The code will look like this:

```rust
use anyhow::Result;

fn my_func() -> Result<()> {
    let x = some_other_func()?;    
    Ok(())
}
```

If you _don't_ use `anyhow`, and the two error types are incompatible, you will have to "box" the error type:

```rust
use std::err::Error;

fn my_func() -> Result<(), Box<dyn Error>> {
    let x = some_other_func()?;      
    Ok(())
}
```

My suggestion: just use `anyhow`.

Later in your journey when you start making crates for other people to use, it can be useful to use the `thiserror` crate rather than the `anyhow` crate. `thiserror` provides conveniences for creating specific named error types that consumers of your crate can use. Both `anyhow` and `thiserror` are made by the same person, David Tolnay.

## Default build configuration

If you're building on Linux, you can create this file to alter
the default settings for compiling programs:

```toml
# ~/.cargo/config.toml
[target.x86_64-unknown-linux-gnu]
rustflags = [
    "-C", "link-arg=-fuse-ld=lld",
]

[profile.release]
lto = "fat"
codegen-units = 1

[profile.dev.package."*"]
opt-level = 3
```

The `rustflags` setting tells cargo to use the much faster `lld` linker. The default linker is `ld`. By default, `lld` will not be installed; you must install it with `sudo apt install lld`, or whatever your package manager requires.

The `profile.release` section enables full "link time optimization" (LTO) which gives the linker more opportunity for applying performance optimizations. Likewise the `codegen-units` also allows the possibility of additional optimizations. Both of these will be at the expense of compile time so these settings should only be used when building a final release.

The final setting, `profile.dev.package."*"`, tells cargo to build your *dependencies* in release mode, even when you're building your dev package in debug mode. The main benefit of this is that your debug build can still run quite fast, depending on what it's doing. This can sometimes dramatically help performance in interactive applications like games, generative art and similar applications.

Note that if necessary you can override these and any other settings by creating a file `./.cargo/config.toml` inside any of your cargo projects.

## Setting up logging

The `log` create provides these macros for producing log messages from your program according to different logging levels. For example:

```rust
use log::*;

fn main() {
    trace!("This will only print at debug level");
    debug!("This will only print at debug level");
    info!("This is info lvl, here's a param: {}", 123);
    warn!("Warning");
    error!("Error level");
}
```

That's pretty much all the `log` crate provides, and it's up to other crates to provide ways to configure the logging level, or where to send logs.

The simplest option I've found so far is the `stderrlog` package. It provides easy configuration options to set up logging, and it plays nicely with `structlog` and `paw`, which I discuss in the next section.

## Command-line parameters with paw

There is a library called [paw]() that makes it very, very easy to add robust support to your CLI applications for command-line parameters. It also integrates with another library `structopt` which allows command-line parameters to be defined declaratively.  Also, in the example below I show also how to set up the `stderrlog` crate from options provided on the command line.

This is how you quickly get set up (4 steps):

1. `$ cargo add paw`
2. `$ cargo add structlog`
3. Now edit your `Cargo.toml` by hand: change the `structlog` entry to this (but use the same version as what you got from the previous command):
```toml
[dependencies]
structopt = { version = "<x.y.z>", features = ["paw"] }
```

4. Edit your `main.rs` file to look something like this:

```rust
use log::*;

#[derive(structopt::StructOpt)]
#[structopt()]
struct Args {
    /// Silence all output
    #[structopt(short = "q", long = "quiet")]
    quiet: bool,
    /// Verbose mode (-v, -vv, -vvv, etc)
    #[structopt(short = "v", long = "verbose", parse(from_occurrences))]
    verbose: usize,
    /// Timestamp (sec, ms, ns, none)
    #[structopt(short = "t", long = "timestamp")]
    ts: Option<stderrlog::Timestamp>,
    /// All your other stuff goes here, e.g.
    #[structopt(parse(from_os_str), short = "o", long = "output-dir")]
    output_dir: Option<std::path::PathBuf>,
}

#[paw::main]
fn main(args: Args) -> {
    stderrlog::new()
        .module(module_path!())
        .quiet(args.quiet)
        .verbosity(args.verbose)
        .timestamp(args.ts.unwrap_or(stderrlog::Timestamp::Off))
        .init()
        .unwrap();

    // All the rest of your code goes here. For example:
    println!("This was the output_dir: {:?}", &args.output_dir);
}
```

Your program can now be called, for example, like this:

```shell
$ myprog -vv --timestamp=ms
```

and you will have millisecond timestamps on your log messages, which will be logged at "warning" level. If you wanted to log at "info" level, you would specify `-vvv`, and so on.